# The Battle of Neighborhoods (Final Report)
### Introduction

Business Problem:
The challenge is to find a suitable apartment for rent near Hyderabad that complies with the demands on location, price and venues. The data required to resolve this challenge is described in the following section 2, below.

### 2.Data Section
The following data is required to answer the issues of the problem:

1.List of Boroughs and neighborhoods of secunderabad with their geodata (latitude and longitude)

2.List of Subway metro stations in secunderabad with their address location

3.List of apartments for rent in secunderabad area with their addresses and price

4.Preferably, a list of apartment for rent with additional information, such as price, address, area, # of beds, etc

5.Venues for each secunderabad neighborhood ( than can be clustered)

6.Venues for subway metro stations, as needed

In [1]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
neighborhood_latitude=17.434227
neighborhood_longitude=78.386659

In [4]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
CLIENT_ID = 'VWREFNGMYCW5LX54RV42VS4G1U1Z4QJ55JY3NXAZXCRGYNUS' # your Foursquare ID
CLIENT_SECRET = '555Q0NH0RPQ5LVZPZ2Q1JBOYGRBBNKY4O222GRATAPCQQFQS' # your Foursquare Secret
VERSION = '20180604'
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=VWREFNGMYCW5LX54RV42VS4G1U1Z4QJ55JY3NXAZXCRGYNUS&client_secret=555Q0NH0RPQ5LVZPZ2Q1JBOYGRBBNKY4O222GRATAPCQQFQS&v=20180604&ll=17.434227,78.386659&radius=500&limit=100'

In [5]:
results = requests.get(url).json()

In [6]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [8]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Inorbit Mall,Shopping Mall,17.433610,78.386207
1,Cream Stone Creations,Ice Cream Shop,17.434191,78.386829
2,Chutneys,Vegetarian / Vegan Restaurant,17.434254,78.386864
3,Fusion 9,Lounge,17.434292,78.386714
4,Food Court @ Inorbit Mall,Food Court,17.434199,78.386831
5,KFC,Fast Food Restaurant,17.434201,78.386809
6,PVR Cinemas,Movie Theater,17.434245,78.386763
7,Starbucks,Coffee Shop,17.433429,78.386605
8,News Cafe,Cocktail Bar,17.434421,78.386998
9,smaaash,Laser Tag,17.433648,78.386430


In [12]:
address = 'secunderabad,IN'

geolocator = Nominatim(user_agent='http')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Secundrabad(Hyderabad),India are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Secundrabad(Hyderabad),India are 17.4337246, 78.5006827.


In [14]:
map_hyd = folium.Map(location=[latitude, longitude], zoom_start=17)

# add markers to map
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_hyd)  
    
map_hyd